# EdThrive data extraction
The objective of this process is to replicate the data extraction needed to build the **Student Engagement Report**. Given the difficulties observed with the current data processing algorithm and the inconsistencies across the expected and actual outputs, it was decided to deep dive into the current algorithm to identify individual operations applied to the data, and replicate them in a more efficient way.

For this purpose, Python was the language of choice to transform the data. 

### Considerations
*   Most of the operations here are done by using only the record IDs from specific tables in the database
*   Identifiers are converted to human understandable information as a last step in the transformations
*   Specific details about the calculations or language specific sintax are not part of the comments on this notebook.
*   Only the relevant columns are loaded from the raw tables on the database

## Data loading
Data was extracted raw from the database, so a series of transformations, joins and calculations are needed to be applied for making analysis. For an easier process of loading the relevant data, some functions were defined.

In [1]:
#Import modules
import pandas as pd
import numpy as np

#Define utils functions
def loader(filename, columns = None):
    filepath = 'Data/{}.csv'
    df = pd.read_csv(filepath.format(filename))
    df = df[columns] if columns != None else df
    return df

def merger(df_left, df_right, on_left, on_right, how = 'inner', drop_keys = 'none', suffix = ('_left', '_right')):
    merged = df_left.merge(df_right, how = how, left_on = on_left, right_on = on_right, suffixes = suffix)
    if drop_keys == 'both': merged.drop(columns=[on_left, on_right], inplace = True)
    if drop_keys == 'right': merged.drop(columns=[on_right], inplace = True)
    if drop_keys == 'left': merged.drop(columns=[on_left], inplace = True)

    return merged

## Observations data (Coaching table)
The main data needed for reporting purposes comes from a table called *Coaching*. In the site, there are filters applied to this data when the report page is open that restrict the results being rendered. The most relevant filters are:
*   Only the results from the last month with available data are shown in the plots.
*   Organization is filtered to the subdomain on which the report is consulted.
*   The observations are restricted to the *observestudentengage* value on the type column.

These filters are not used in this process because they are either irrelevant or redundant to the data being processed.

In [2]:
#Load global observations data
coaching = loader('Coaching', ['id','startTime','status','orgId','state','whoId','coachId','type','numberOfStudents'])
coaching['startTime'] = pd.to_datetime(coaching['startTime'], format='%Y-%m-%d %H:%M:%S')
coaching['month'] = coaching['startTime'].dt.month
coaching.head()

,id,startTime,status,orgId,state,whoId,coachId,type,numberOfStudents,month
0,4162,2022-03-07 20:57:00,delete,1000163,concluded,3807,1259,observestudentengage,20,3
1,4163,2022-03-07 21:08:56,delete,1000163,concluded,3807,3739,observestudentengage,20,3
2,4164,2022-03-07 21:09:01,delete,1000163,concluded,3808,3745,observestudentengage,20,3
3,4165,2022-03-07 21:09:22,delete,1000163,concluded,3779,3395,observestudentengage,20,3
4,4172,2022-03-08 16:33:21,delete,1000163,concluded,3807,1259,observestudentengage,20,3


The previous table is the body to which the different engagement variables is going to be added, by it's identifier *id*.

## Engagement activities (Engagement table)
The engagement activities are the group of variables that are measured and presented as the **Student Engagement**, **Learning Tasks**, **Student Grouping** plots in the report. 

In [3]:
#Load engagement types
engagement = loader('CoachingEngagement',['id','name','type'])
engagement.rename(columns={'id':'engagementId','name':'engagementName','type':'engagementType'},inplace=True)
engagement

,engagementId,engagementName,engagementType
0,1,Active,engagement
1,2,Passive,engagement
2,3,Off-Task,engagement
3,4,Down Time,engagement
4,5,Listening,learningtask
5,6,Doing/Demonstrating,learningtask
6,7,Reading,learningtask
7,8,Speaking,learningtask
8,9,Writing,learningtask
9,10,None,learningtask


## Observations details (Coaching Data table)
The observations details are stored in a table called CoachingData. For each user-app interacction on the Observation scren, several records are saved on the database, one for each user-app interaction. For example, in the image below, one record was created for the user selecting **Off-Task - Engagement**, one for **Listening - Learning Tasks**, and one for **Small Group - Student Grouping**, and so on with the other variables. All these records are related to the Coaching table through a *coaching Id*.

![Student Engagement Observation](Images/StudentEngagementObservation.png)

In [4]:
#Load coachingData
coachingData = loader('CoachingData', ['status', 'coachingId', 'engagementId','userId','studentNum', 'type', 'note', 'count'])
coachingData.head()

,status,coachingId,engagementId,userId,studentNum,type,note,count
0,delete,4162,2.0,1259,1.0,engagement,NaN,NaN
1,delete,4162,5.0,1259,1.0,engagement,NaN,NaN
2,delete,4162,11.0,1259,1.0,engagement,NaN,NaN
3,delete,4162,1.0,1259,2.0,engagement,NaN,NaN
4,delete,4162,8.0,1259,2.0,engagement,NaN,NaN


#### Number of observations
Two important variables from the report are **Total observations** and **Total students**. The latter can be found in the Coaching table with the name *numberOfStudents*. However, the first one needs to be calculated from the amount of records counted by the varialbe *studentNum* on the *coachingData* table, for each Observation session. This new variable is appended to the **coaching** table with the name of *totalObservations*.

In [5]:
#Calculate totalObservations
coachingDataTotalObs = coachingData[['coachingId','studentNum']]
coachingDataTotalObs = coachingDataTotalObs.groupby('coachingId').max('studentNum')
coachingDataTotalObs.reset_index(inplace=True)
coaching = merger(coaching, coachingDataTotalObs, 'id', 'coachingId', how = 'left', drop_keys = 'right')
coaching.rename(columns={'studentNum':'totalObservations'}, inplace=True)
coaching.head()

,id,startTime,status,orgId,state,whoId,coachId,type,numberOfStudents,month,totalObservations
0,4162,2022-03-07 20:57:00,delete,1000163,concluded,3807,1259,observestudentengage,20,3,3.0
1,4163,2022-03-07 21:08:56,delete,1000163,concluded,3807,3739,observestudentengage,20,3,12.0
2,4164,2022-03-07 21:09:01,delete,1000163,concluded,3808,3745,observestudentengage,20,3,17.0
3,4165,2022-03-07 21:09:22,delete,1000163,concluded,3779,3395,observestudentengage,20,3,15.0
4,4172,2022-03-08 16:33:21,delete,1000163,concluded,3807,1259,observestudentengage,20,3,7.0


#### Student Engagement, Learning Task, Student Grouping
Next, the variables corresponding to the images below were added. In the table *coachingData*, these variables are stored in a disaggregated and narrow form ([see more about data forms here](https://en.wikipedia.org/wiki/Wide_and_narrow_data)). It was necessary aggregate and reshape the data to fit with the previous *coaching* table.

![Student Engagement](Images/StudentEngagement.png)
![Learning Tasks](Images/LearningTasks.png)
![Student Grouping](Images/StudentGrouping.png)

In [6]:
#Select columns to use and calculate total observations per variable
coachingDataEngagement = coachingData[['coachingId','engagementId','status']].groupby(['coachingId','engagementId']).count()
coachingDataEngagement.reset_index(inplace=True)

#Combine data with engagement to get variable labels
coachingDataEngagement = merger(coachingDataEngagement, engagement, 'engagementId','engagementId')

#Reshape table to have a column per variable and add zeros in cells with no values
coachingDataEngagement = coachingDataEngagement.pivot(index='coachingId', columns='engagementName',values='status')
coachingDataEngagement.reset_index(inplace=True)
coachingDataEngagement.fillna(0, inplace=True)
coachingDataEngagement.head()

engagementName,coachingId,Active,Doing/Demonstrating,Down Time,Independent Work Session,Listening,No Grouping,None,Off-Task,Partner,Passive,Reading,Small Group,Speaking,Whole Class,Writing
0,4162,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0
1,4163,7.0,3.0,0.0,0.0,3.0,0.0,1.0,1.0,0.0,4.0,0.0,5.0,4.0,7.0,1.0
2,4164,8.0,0.0,2.0,0.0,5.0,0.0,0.0,3.0,4.0,4.0,0.0,0.0,5.0,13.0,7.0
3,4165,8.0,0.0,0.0,5.0,6.0,0.0,0.0,2.0,4.0,5.0,0.0,0.0,4.0,6.0,5.0
4,4172,3.0,5.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,3.0,1.0,3.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,4634,23.0,41.0,1.0,41.0,0.0,0.0,0.0,3.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0
207,4635,31.0,30.0,0.0,30.0,5.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,5.0,0.0
208,4637,26.0,3.0,2.0,1.0,5.0,0.0,0.0,6.0,23.0,5.0,7.0,15.0,24.0,0.0,0.0
209,4638,13.0,0.0,8.0,0.0,19.0,0.0,1.0,0.0,0.0,17.0,0.0,0.0,0.0,41.0,21.0


The data above is the disaggregated data required for the plots shown before. Adding this variables to the *coaching* table, it results in the following.

In [7]:
coaching = merger(coaching, coachingDataEngagement, 'id', 'coachingId', how = 'left', drop_keys = 'right')
coaching.head()

,id,startTime,status,orgId,state,whoId,coachId,type,numberOfStudents,month,...,No Grouping,None,Off-Task,Partner,Passive,Reading,Small Group,Speaking,Whole Class,Writing
0,4162,2022-03-07 20:57:00,delete,1000163,concluded,3807,1259,observestudentengage,20,3,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0
1,4163,2022-03-07 21:08:56,delete,1000163,concluded,3807,3739,observestudentengage,20,3,...,0.0,1.0,1.0,0.0,4.0,0.0,5.0,4.0,7.0,1.0
2,4164,2022-03-07 21:09:01,delete,1000163,concluded,3808,3745,observestudentengage,20,3,...,0.0,0.0,3.0,4.0,4.0,0.0,0.0,5.0,13.0,7.0
3,4165,2022-03-07 21:09:22,delete,1000163,concluded,3779,3395,observestudentengage,20,3,...,0.0,0.0,2.0,4.0,5.0,0.0,0.0,4.0,6.0,5.0
4,4172,2022-03-08 16:33:21,delete,1000163,concluded,3807,1259,observestudentengage,20,3,...,0.0,0.0,1.0,3.0,3.0,1.0,3.0,0.0,1.0,0.0
